# Sistemas Lineares (com exemplo 4x4)

Este notebook apresenta:
- Conceitos básicos de sistemas lineares \(A\,x = b\)
- Método da eliminação de Gauss com pivotamento parcial (implementação própria)
- Comparação com `numpy.linalg.solve`
- Classificação via posto, determinante e número de condição
- Exemplos de sistema sem solução e com infinitas soluções

Requisitos: Python 3.9+ e `numpy`.



In [ ]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)

TOL = 1e-12  # tolerância para teste de pivô nulo



In [ ]:
def forward_elimination_with_partial_pivot(A: np.ndarray, b: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    A = A.astype(float).copy()
    b = b.astype(float).copy()
    n = A.shape[0]
    
    for k in range(n - 1):
        # Pivotamento parcial: escolhe maior |A[i,k]| para i >= k
        pivot_row = k + np.argmax(np.abs(A[k:, k]))
        if np.abs(A[pivot_row, k]) < TOL:
            # pivô praticamente zero, matriz singular ou quase singular
            continue
        if pivot_row != k:
            A[[k, pivot_row], :] = A[[pivot_row, k], :]
            b[[k, pivot_row]] = b[[pivot_row, k]]
        
        for i in range(k + 1, n):
            if np.abs(A[k, k]) < TOL:
                continue
            m = A[i, k] / A[k, k]
            A[i, k:] -= m * A[k, k:]
            b[i] -= m * b[k]
    
    return A, b


def back_substitution(U: np.ndarray, c: np.ndarray) -> np.ndarray:
    n = U.shape[0]
    x = np.zeros(n, dtype=float)
    for i in range(n - 1, -1, -1):
        if np.abs(U[i, i]) < TOL:
            # Sem pivô válido nesta linha
            if np.abs(c[i]) < TOL:
                # 0 = 0 -> grau de liberdade (infinitas soluções)
                x[i] = 0.0
                continue
            else:
                raise np.linalg.LinAlgError("Sistema inconsistente ou matriz singular")
        s = np.dot(U[i, i + 1:], x[i + 1:])
        x[i] = (c[i] - s) / U[i, i]
    return x


def gauss_solve(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    if A.ndim != 2 or A.shape[0] != A.shape[1]:
        raise ValueError("A deve ser uma matriz quadrada")
    if b.ndim != 1 or b.shape[0] != A.shape[0]:
        raise ValueError("b deve ser um vetor coluna compatível com A")
    U, c = forward_elimination_with_partial_pivot(A, b)
    return back_substitution(U, c)



In [ ]:
# Exemplo 4x4 com solução única
A = np.array([
    [2., -1.,  0.,  3.],
    [1.,  0.,  4., -2.],
    [0.,  5., -1.,  1.],
    [3.,  2.,  1.,  0.]
])

x_true = np.array([1., 2., -1., 3.])
b = A @ x_true

x_gauss = gauss_solve(A, b)
x_numpy = np.linalg.solve(A, b)

res_gauss = np.linalg.norm(A @ x_gauss - b)
err_gauss = np.linalg.norm(x_gauss - x_true)

print("A =\n", A)
print("b =", b)
print("x (Gauss)    =", x_gauss)
print("x (NumPy)     =", x_numpy)
print("resíduo (Gauss) =", res_gauss)
print("erro vs. x_true =", err_gauss)



In [ ]:
# Classificação: posto, determinante e condição
rank_A = np.linalg.matrix_rank(A)
det_A = np.linalg.det(A)
rank_Ab = np.linalg.matrix_rank(np.c_[A, b])
cond_A = np.linalg.cond(A)

print("posto(A)  =", rank_A)
print("posto([A|b]) =", rank_Ab)
print("det(A)    =", det_A)
print("cond(A)   =", cond_A)

if rank_A == rank_Ab == A.shape[0]:
    print("=> Solução única")
elif rank_A == rank_Ab < A.shape[0]:
    print("=> Infinitas soluções")
else:
    print("=> Sistema inconsistente (sem solução)")



In [ ]:
# Exemplo de sistema inconsistente (sem solução)
A_inc = np.array([
    [1., 1., 0., 0.],
    [0., 1., 1., 0.],
    [1., 2., 1., 0.],  # combinação linear das duas primeiras
    [0., 0., 0., 1.]
])

b_inc = np.array([1., 1., 3., 0.])  # torna [A|b] de posto maior que posto(A)

print("posto(A_inc)   =", np.linalg.matrix_rank(A_inc))
print("posto([A_inc|b_inc]) =", np.linalg.matrix_rank(np.c_[A_inc, b_inc]))

try:
    x_inc = gauss_solve(A_inc, b_inc)
    print("x_inc =", x_inc)
except Exception as e:
    print("Falha ao resolver (como esperado):", e)



In [ ]:
# Exemplo de sistema com infinitas soluções
A_inf = np.array([
    [1., 0., 0., 0.],
    [0., 1., 0., 0.],
    [0., 0., 1., 0.],
    [1., 1., 1., 0.]  # dependente das três primeiras
])

b_inf = np.array([1., 2., 3., 6.])  # compatível com a dependência

rank_A_inf = np.linalg.matrix_rank(A_inf)
rank_Ab_inf = np.linalg.matrix_rank(np.c_[A_inf, b_inf])
print("posto(A_inf)   =", rank_A_inf)
print("posto([A_inf|b_inf]) =", rank_Ab_inf)

# Usamos solução de mínima norma via mínimos quadrados
x_inf, *_ = np.linalg.lstsq(A_inf, b_inf, rcond=None)
print("uma solução (mínima norma) =", x_inf)

# Verifica
print("resíduo =", np.linalg.norm(A_inf @ x_inf - b_inf))



In [ ]:
# Função utilitária: gera e resolve um sistema aleatório bem condicionado
rng = np.random.default_rng(42)

def sistema_aleatorio_4x4(seed: int | None = None):
    r = np.random.default_rng(seed)
    A = r.normal(size=(4, 4))
    # reforça diagonal dominante para garantir unicidade
    for i in range(4):
        A[i, i] += 4.0
    x_true = r.normal(size=4)
    b = A @ x_true
    return A, b, x_true

A_r, b_r, x_r_true = sistema_aleatorio_4x4(123)
x_r = gauss_solve(A_r, b_r)
print("||Ax-b|| =", np.linalg.norm(A_r @ x_r - b_r))
print("||x - x_true|| =", np.linalg.norm(x_r - x_r_true))

